In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_core_range_assortment_wmt
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT
    store_id,
    product_id,
    core_range,
    upload_user,
    upload_time
FROM (
    SELECT *
    FROM {catalog_name}.udp_wcm_silver_dataportal.d_core_range_assortment_wmt
    WHERE DATE(proc_date) >= '2024-09-24'
    QUALIFY ROW_NUMBER() OVER (
        PARTITION BY store_id, product_id
        ORDER BY proc_date DESC, upload_time DESC
    ) = 1
)
WHERE IFNULL(TRIM(UPPER(delete_indicator)), 'NULL') NOT IN ('YES', 'X')
""")